In [1]:
import pandas as pd
import numpy as np
import torch
import cv2
import mediapipe as mp
import requests

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [2]:
words_corpus_df = pd.read_excel("ISL_CSLRT_Corpus_word_details.xlsx")

In [3]:
words_corpus_df.head()

,Word,Frames path
0,A LOT,ISL_CSLRT_Corpus\Frames_Word_Level\A LOT\A LOT...
1,A LOT,ISL_CSLRT_Corpus\Frames_Word_Level\A LOT\A LOT...
2,A LOT,ISL_CSLRT_Corpus\Frames_Word_Level\A LOT\A LOT...
3,A LOT,ISL_CSLRT_Corpus\Frames_Word_Level\A LOT\A LOT...
4,ABUSE,ISL_CSLRT_Corpus\Frames_Word_Level\ABUSE\ABUSE...


In [5]:
words_corpus_df.shape

(1036, 2)

In [7]:
words_corpus_df['Frames path'] = words_corpus_df['Frames path'].str.replace(r'^ISL_CSLRT_Corpus\\', '', regex=True)
## we don't need ISL_CSLRT as it is not part of path

frame_path = words_corpus_df.iloc[2][1]

In [8]:
file_url = r"C:\Users\hp\Indian-Sign-Language-SIH\dataset"

In [9]:
full_path = fr"{file_url}{frame_path}"
full_path

'C:\\Users\\hp\\Indian-Sign-Language-SIH\\datasetFrames_Word_Level\\A LOT\\A LOT (3).jpg'

In [11]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

def extract_features(hand_landmarks):
    # Extracting the x, y, z coordinates of the landmarks
    features = []
    for landmark in hand_landmarks.landmark:
        features.append(landmark.x)
        features.append(landmark.y)
        features.append(landmark.z)

    # Convert list to a numpy array (flatten the landmarks)
    features = np.array(features)
    
    # Optionally, calculate distances between specific landmarks, e.g., thumb to index tip
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    distance_thumb_index = np.sqrt((thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 + (thumb_tip.z - index_tip.z)**2)
    features = np.append(features, distance_thumb_index)

    return features

In [12]:
# mp_hands = mp.solutions.hands
# hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

# Dictionary to store features with indexes from `words_corpus_df`
features_dict = {}

for i in range(len(words_corpus_df)):
    frame_path = words_corpus_df.iloc[i][1]
    full_path = rf"{file_url}\{frame_path}"

    # Read the image with the correct full path
    image = cv2.imread(full_path)

    if image is not None:
        # Convert the image to RGB as MediaPipe works with RGB images
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        # Process the image with MediaPipe
        results = hands.process(image_rgb)
    
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Extract features from each detected hand
                features = extract_features(hand_landmarks)
                # Store the features with the index
                features_dict[i] = features
    
        else:
            print(f"No hand landmarks detected in the image at index {i}")

# Convert the dictionary to a DataFrame to retain original indexes
features_df = pd.DataFrame.from_dict(features_dict, orient='index')

# Release MediaPipe resources
hands.close()


No hand landmarks detected in the image at index 24
No hand landmarks detected in the image at index 51
No hand landmarks detected in the image at index 53
No hand landmarks detected in the image at index 54
No hand landmarks detected in the image at index 117
No hand landmarks detected in the image at index 127
No hand landmarks detected in the image at index 135
No hand landmarks detected in the image at index 147
No hand landmarks detected in the image at index 186
No hand landmarks detected in the image at index 206
No hand landmarks detected in the image at index 217
No hand landmarks detected in the image at index 238
No hand landmarks detected in the image at index 247
No hand landmarks detected in the image at index 277
No hand landmarks detected in the image at index 311
No hand landmarks detected in the image at index 313
No hand landmarks detected in the image at index 316
No hand landmarks detected in the image at index 322
No hand landmarks detected in the image at index 3

In [14]:
print(features_df.shape)

(984, 64)


In [16]:
features_df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.316836,0.439745,7.139941e-08,0.322064,0.408899,-0.012561,0.320306,0.373865,-0.020178,0.318748,...,0.260467,0.404656,-0.028454,0.251036,0.396338,-0.031925,0.241957,0.388138,-0.034238,0.061003
1,0.777162,0.594822,1.888133e-07,0.766787,0.569503,-0.029101,0.766939,0.543866,-0.046024,0.770992,...,0.831578,0.531044,-0.019791,0.839598,0.514735,-0.025115,0.845963,0.498970,-0.028685,0.066329
2,0.282781,0.426264,9.168880e-08,0.279939,0.391712,0.005966,0.275245,0.354410,0.005695,0.274172,...,0.289899,0.329482,-0.034678,0.295946,0.304782,-0.038501,0.301991,0.284427,-0.040737,0.070718
3,0.613345,0.728356,7.959181e-08,0.623554,0.698023,-0.007444,0.642625,0.672808,-0.011870,0.657584,...,0.660544,0.768533,-0.024703,0.665910,0.764937,-0.027244,0.669510,0.756921,-0.028706,0.074697
4,0.493275,0.434260,8.329591e-08,0.508639,0.422977,-0.002672,0.521094,0.407595,-0.007269,0.529103,...,0.498473,0.365121,-0.024746,0.509517,0.378327,-0.025282,0.514609,0.389408,-0.024399,0.016817


In [21]:
features_df.tail()
# as last index is as same as as original so we can work with it

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
1031,0.516384,0.934833,1.200309e-07,0.502227,0.941857,-0.007903,0.490989,0.966455,-0.009668,0.483381,...,0.500649,1.025498,0.003856,0.493630,1.035574,0.003876,0.488021,1.041036,0.004341,0.040912
1032,0.466291,0.499750,4.077212e-08,0.490007,0.485130,-0.007605,0.509123,0.470275,-0.015650,0.514297,...,0.458378,0.503234,-0.036172,0.463337,0.510462,-0.033446,0.462120,0.504638,-0.031439,0.100023
1033,0.438552,0.430639,2.417242e-08,0.452326,0.422841,-0.009162,0.466148,0.410837,-0.017689,0.470169,...,0.414486,0.411160,-0.033348,0.425129,0.428160,-0.030595,0.431474,0.434662,-0.027218,0.106812
1034,0.188927,0.414196,3.298646e-07,0.217849,0.404225,-0.019165,0.263034,0.383194,-0.028543,0.296865,...,0.244951,0.375255,-0.057221,0.238747,0.387291,-0.054817,0.228009,0.397775,-0.048664,0.076164
1035,0.467205,0.484764,1.373348e-07,0.478713,0.443635,0.000470,0.498948,0.413178,-0.005146,0.518559,...,0.513509,0.461524,-0.031306,0.508589,0.467270,-0.025416,0.498717,0.466678,-0.020911,0.058348


In [23]:
word_column_df = words_corpus_df[['Word']]

combined_df = pd.concat([word_column_df, features_df], axis=1)

combined_df.head()

,Word,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,A LOT,0.316836,0.439745,7.139941e-08,0.322064,0.408899,-0.012561,0.320306,0.373865,-0.020178,...,0.260467,0.404656,-0.028454,0.251036,0.396338,-0.031925,0.241957,0.388138,-0.034238,0.061003
1,A LOT,0.777162,0.594822,1.888133e-07,0.766787,0.569503,-0.029101,0.766939,0.543866,-0.046024,...,0.831578,0.531044,-0.019791,0.839598,0.514735,-0.025115,0.845963,0.498970,-0.028685,0.066329
2,A LOT,0.282781,0.426264,9.168880e-08,0.279939,0.391712,0.005966,0.275245,0.354410,0.005695,...,0.289899,0.329482,-0.034678,0.295946,0.304782,-0.038501,0.301991,0.284427,-0.040737,0.070718
3,A LOT,0.613345,0.728356,7.959181e-08,0.623554,0.698023,-0.007444,0.642625,0.672808,-0.011870,...,0.660544,0.768533,-0.024703,0.665910,0.764937,-0.027244,0.669510,0.756921,-0.028706,0.074697
4,ABUSE,0.493275,0.434260,8.329591e-08,0.508639,0.422977,-0.002672,0.521094,0.407595,-0.007269,...,0.498473,0.365121,-0.024746,0.509517,0.378327,-0.025282,0.514609,0.389408,-0.024399,0.016817


In [24]:
combined_df.shape

(1036, 65)

In [26]:
combined_df.isna().sum()

Word     0
0       52
1       52
2       52
3       52
        ..
59      52
60      52
61      52
62      52
63      52
Length: 65, dtype: int64

In [27]:
combined_df.dropna(axis=0, inplace=True)

In [28]:
combined_df.isna().sum()

Word    0
0       0
1       0
2       0
3       0
       ..
59      0
60      0
61      0
62      0
63      0
Length: 65, dtype: int64

In [30]:
combined_df.shape

(984, 65)

In [31]:
cleaned_df = combined_df.copy()

In [34]:
# Saving cleaned_df to a CSV file
# preserving the indexes
cleaned_df.to_csv('words_data.csv', index=True)